In [1]:
#Importing dataset
from google.colab import files
uploaded = files.upload()

Saving d.csv to d.csv


In [2]:
import io
import pandas as pd
df = pd.read_csv(io.BytesIO(uploaded['d.csv']))
# Dataset is now stored in a Pandas Dataframe

In [3]:
#Importing necessary Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
plt.style.use('fivethirtyeight') # For plots

In [4]:
#Taing log of volume (This is a technique which is used for enabling model to predict only positive values)
df['volume']=np.log(df['volume'])

In [5]:
#Create another datafarme
#Prophet only understands the DateTime column as 'ds' & target column as 'y', so rename it as same

df2=pd.DataFrame()
df2['ds']=df['DateTime']
df2['y']=df['volume']
df2

,ds,y
0,2017-04-01 00:00:00,2.302585
1,2017-04-01 01:00:00,2.484907
2,2017-04-01 02:00:00,1.609438
3,2017-04-01 03:00:00,1.609438
4,2017-04-01 04:00:00,2.833213
...,...,...
11563,2018-07-31 19:00:00,6.300786
11564,2018-07-31 20:00:00,6.773080
11565,2018-07-31 21:00:00,5.955837
11566,2018-07-31 22:00:00,5.407172


In [6]:
#Train the model till June 
train=df2.iloc[:10826]
train

,ds,y
0,2017-04-01 00:00:00,2.302585
1,2017-04-01 01:00:00,2.484907
2,2017-04-01 02:00:00,1.609438
3,2017-04-01 03:00:00,1.609438
4,2017-04-01 04:00:00,2.833213
...,...,...
10821,2018-06-30 19:00:00,6.590301
10822,2018-06-30 20:00:00,6.694562
10823,2018-06-30 21:00:00,5.940171
10824,2018-06-30 22:00:00,5.298317


In [7]:
#Testing dataset for july
test=df2.iloc[10826:]
test

,ds,y
10826,2018-07-01 00:00:00,3.912023
10827,2018-07-01 01:00:00,3.367296
10828,2018-07-01 02:00:00,1.386294
10829,2018-07-01 03:00:00,0.693147
10830,2018-07-01 04:00:00,3.091042
...,...,...
11563,2018-07-31 19:00:00,6.300786
11564,2018-07-31 20:00:00,6.773080
11565,2018-07-31 21:00:00,5.955837
11566,2018-07-31 22:00:00,5.407172


In [8]:
#Creating model
#If you want to enable the following seasonality you can enable or disable it, otherwise go for Auto
m=Prophet(interval_width=0.95,yearly_seasonality='auto',weekly_seasonality='auto',daily_seasonality='auto')
m=Prophet()

#Prophet has an inbuilt function wherein we can pass the holidays as a feature of any desired country
m.add_country_holidays(country_name='IN')

In [17]:
m.train_holiday_names

0                        Republic Day
1                    Independence Day
2                      Gandhi Jayanti
3                              Diwali
4                                Holi
5                       Day of Ashura
6                              Mawlid
7                         Eid al-Fitr
8              Feast of the Sacrifice
9                      New Year's Day
10                        Palm Sunday
11                    Maundy Thursday
12                        Good Friday
13                      Easter Sunday
14                 Feast of Pentecost
15    Fest of St. Theresa of Calcutta
16        Feast of the Blessed Virgin
17                     All Saints Day
18                      All Souls Day
19                      Christmas Day
20                         Boxing Day
21               Feast of Holy Family
dtype: object

In [9]:
#fit the model on training dataset
m.fit(train)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
/usr/local/lib/python3.6/dist-packages/fbprophet/hdays.py:259: Warning:

We only support Diwali and Holi holidays from 2010 to 2030



In [10]:
m.params

{'beta': array([[-0.00174683,  0.01483714, -0.00188707, -0.01622422,  0.00364626,
          0.00894004, -0.15440367, -0.28091055, -0.08719931, -0.03677458,
          0.00345534,  0.00970945,  0.00117602,  0.00926138,  0.02937154,
          0.05046216, -0.04152021, -0.04201948,  0.01154895, -0.02772711,
          0.00633688, -0.02593293,  0.01572453,  0.00606169,  0.03999215,
          0.03765594,  0.01632687, -0.05769864, -0.03101103, -0.01675806,
         -0.05185743, -0.00711022,  0.01071851,  0.02678469, -0.01772531,
         -0.04404241]]),
 'delta': array([[ 2.14150394e-01,  7.53809785e-02,  3.45368711e-02,
          3.13411754e-06, -1.13626875e-07, -1.05419086e-06,
          2.21472170e-07,  3.97587686e-06,  3.52893881e-03,
          5.66362894e-02,  2.28663721e-06,  9.35251121e-07,
         -4.10547438e-01,  5.18756396e-04,  8.91018580e-03,
          2.91698958e-01,  6.47508569e-07,  7.95598928e-07,
          3.73894712e-07,  1.93169228e-06, -5.49218734e-07,
          3.80757715

In [11]:
#Making future dataframe
#Here we have to forecast values for July, therefore 31days x 24hrs= 744hrs

future=m.make_future_dataframe(periods=744,freq='H',include_history=False)
future

,ds
0,2018-07-01 00:00:00
1,2018-07-01 01:00:00
2,2018-07-01 02:00:00
3,2018-07-01 03:00:00
4,2018-07-01 04:00:00
...,...
739,2018-07-31 19:00:00
740,2018-07-31 20:00:00
741,2018-07-31 21:00:00
742,2018-07-31 22:00:00


In [12]:
#Use predict function on the dataset we've just created 
forecast=m.predict(future)
forecast[['ds','yhat','yhat_lower','yhat_upper']].tail()

/usr/local/lib/python3.6/dist-packages/fbprophet/hdays.py:259: Warning:

We only support Diwali and Holi holidays from 2010 to 2030

/usr/local/lib/python3.6/dist-packages/fbprophet/hdays.py:259: Warning:

We only support Diwali and Holi holidays from 2010 to 2030



,ds,yhat,yhat_lower,yhat_upper
739,2018-07-31 19:00:00,6.265207,5.686953,6.848397
740,2018-07-31 20:00:00,5.629273,5.002216,6.200079
741,2018-07-31 21:00:00,4.871610,4.254845,5.500067
742,2018-07-31 22:00:00,4.094135,3.481532,4.707608
743,2018-07-31 23:00:00,3.346353,2.739765,3.982530


In [13]:
#Taking the exponent of the values will bring back the log values to its original form

forecast['yhat']=np.exp(forecast['yhat'])
forecast['yhat_lower']=np.exp(forecast['yhat_lower'])
forecast['yhat_upper']=np.exp(forecast['yhat_upper'])
forecast


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,All Saints Day,All Saints Day_lower,All Saints Day_upper,All Souls Day,All Souls Day_lower,All Souls Day_upper,Boxing Day,Boxing Day_lower,Boxing Day_upper,Christmas Day,Christmas Day_lower,Christmas Day_upper,Day of Ashura,Day of Ashura_lower,Day of Ashura_upper,Diwali,Diwali_lower,Diwali_upper,Easter Sunday,Easter Sunday_lower,Easter Sunday_upper,Eid al-Fitr,Eid al-Fitr_lower,Eid al-Fitr_upper,Feast of Holy Family,Feast of Holy Family_lower,Feast of Holy Family_upper,Feast of Pentecost,Feast of Pentecost_lower,Feast of Pentecost_upper,Feast of the Blessed Virgin,Feast of the Blessed Virgin_lower,Feast of the Blessed Virgin_upper,Feast of the Sacrifice,...,Good Friday,Good Friday_lower,Good Friday_upper,Holi,Holi_lower,Holi_upper,Independence Day,Independence Day_lower,Independence Day_upper,Maundy Thursday,Maundy Thursday_lower,Maundy Thursday_upper,Mawlid,Mawlid_lower,Mawlid_upper,New Year's Day,New Year's Day_lower,New Year's Day_upper,Palm Sunday,Palm Sunday_lower,Palm Sunday_upper,Republic Day,Republic Day_lower,Republic Day_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,holidays,holidays_lower,holidays_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2018-07-01 00:00:00,4.955447,6.077706,21.480659,4.955447,4.955447,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.535837,-2.535837,-2.535837,-2.367790,-2.367790,-2.367790,0.0,0.0,0.0,-0.168048,-0.168048,-0.168048,0.0,0.0,0.0,11.241468
1,2018-07-01 01:00:00,4.955462,3.264087,11.649596,4.955462,4.955462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.133001,-3.133001,-3.133001,-2.947037,-2.947037,-2.947037,0.0,0.0,0.0,-0.185964,-0.185964,-0.185964,0.0,0.0,0.0,6.187068
2,2018-07-01 02:00:00,4.955477,2.430604,8.146648,4.955477,4.955477,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.489224,-3.489224,-3.489224,-3.285882,-3.285882,-3.285882,0.0,0.0,0.0,-0.203342,-0.203342,-0.203342,0.0,0.0,0.0,4.332970
3,2018-07-01 03:00:00,4.955493,2.444376,8.057065,4.955493,4.955493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.459607,-3.459607,-3.459607,-3.239576,-3.239576,-3.239576,0.0,0.0,0.0,-0.220031,-0.220031,-0.220031,0.0,0.0,0.0,4.463289
4,2018-07-01 04:00:00,4.955508,3.746495,13.192056,4.955508,4.955508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.983724,-2.983724,-2.983724,-2.747843,-2.747843,-2.747843,0.0,0.0,0.0,-0.235881,-0.235881,-0.235881,0.0,0.0,0.0,7.183477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,2018-07-31 19:00:00,4.966734,294.993546,942.368940,4.913763,5.013642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [14]:
#Storing the predictions in the 'Results' dataset
results=pd.DataFrame()
results['DateTime']=forecast['ds']
results['predicted']=forecast['yhat']
results

,DateTime,predicted
0,2018-07-01 00:00:00,11.241468
1,2018-07-01 01:00:00,6.187068
2,2018-07-01 02:00:00,4.332970
3,2018-07-01 03:00:00,4.463289
4,2018-07-01 04:00:00,7.183477
...,...,...
739,2018-07-31 19:00:00,525.950440
740,2018-07-31 20:00:00,278.459684
741,2018-07-31 21:00:00,130.530963
742,2018-07-31 22:00:00,59.987401


In [15]:
test['y']=np.exp(test['y'])
test

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,ds,y
10826,2018-07-01 00:00:00,50.0
10827,2018-07-01 01:00:00,29.0
10828,2018-07-01 02:00:00,4.0
10829,2018-07-01 03:00:00,2.0
10830,2018-07-01 04:00:00,22.0
...,...,...
11563,2018-07-31 19:00:00,545.0
11564,2018-07-31 20:00:00,874.0
11565,2018-07-31 21:00:00,386.0
11566,2018-07-31 22:00:00,223.0


In [16]:
import plotly
import plotly.graph_objects as go

# Create random data with numpy
fig = go.Figure()
trace1=go.Scatter(x=results['DateTime'], y=results['predicted'],
                     mode='lines',
                     name='predicted')
trace2=go.Scatter(x=test['ds'], y=test['y'],
                    mode='lines',
                    name='actual')

data=[trace2,trace1]


updatemenus = list([
    dict(active=1,
         buttons=list([
            dict(label='Log Scale',
                 method='update',
                 args=[{'visible': [True, True]},
                       {'title': 'Log scale',
                        'yaxis': {'type': 'log'}}]),
            dict(label='Linear Scale',
                 method='update',
                 args=[{'visible': [True, True]},
                       {'title': 'Linear scale',
                        'yaxis': {'type': 'linear'}}])
            ]),
        )
    ])

layout = dict(updatemenus=updatemenus, title='Linear scale')
fig = go.Figure(data=data, layout=layout)
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(title_text="LSTM UNIVARIATE MODEL")


plotly.offline.iplot(fig)

In [23]:
results

,DateTime,predicted
0,2018-07-01 00:00:00,11.241468
1,2018-07-01 01:00:00,6.187068
2,2018-07-01 02:00:00,4.332970
3,2018-07-01 03:00:00,4.463289
4,2018-07-01 04:00:00,7.183477
...,...,...
739,2018-07-31 19:00:00,525.950440
740,2018-07-31 20:00:00,278.459684
741,2018-07-31 21:00:00,130.530963
742,2018-07-31 22:00:00,59.987401


In [24]:
test

,ds,y
10826,2018-07-01 00:00:00,50.0
10827,2018-07-01 01:00:00,29.0
10828,2018-07-01 02:00:00,4.0
10829,2018-07-01 03:00:00,2.0
10830,2018-07-01 04:00:00,22.0
...,...,...
11563,2018-07-31 19:00:00,545.0
11564,2018-07-31 20:00:00,874.0
11565,2018-07-31 21:00:00,386.0
11566,2018-07-31 22:00:00,223.0
